# Data

In [1]:
import os
import csv
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier

# Setting the path of the training dataset (that was already provided to you)

running_local = True if os.getenv('JUPYTERHUB_USER') is None else False
DATASET_PATH = "."

# Set the location of the dataset
if running_local:
    # If running on your local machine, the sign_lang_train folder's path should be specified here
    local_path = "sign_lang_train"
    if os.path.exists(local_path):
        DATASET_PATH = local_path
else:
    # If running on the Jupyter hub, this data folder is already available
    # You DO NOT need to upload the data!
        DATASET_PATH = "/data/mlproject21/sign_lang_train"
        
# Setting the path of the training dataset (that was already provided to you)

running_local = True if os.getenv('JUPYTERHUB_USER') is None else False
DATASET_PATH = "."

# Set the location of the dataset
if running_local:
    # If running on your local machine, the sign_lang_train folder's path should be specified here
    local_path = "sign_lang_train"
    if os.path.exists(local_path):
        DATASET_PATH = local_path
else:
    # If running on the Jupyter hub, this data folder is already available
    # You DO NOT need to upload the data!
        DATASET_PATH = "/data/mlproject21/sign_lang_train"

# Utility function

def read_csv(csv_file):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, io
from torchvision.utils import make_grid

from string import ascii_lowercase

class SignLangDataset(Dataset):
    """Sign language dataset"""

    def __init__(self, csv_file, root_dir, class_index_map=None, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = read_csv(os.path.join(root_dir,csv_file))
        self.root_dir = root_dir
        self.class_index_map = class_index_map
        self.transform = transform
        # List of class names in order
        self.class_names = list(map(str, list(range(10)))) + list(ascii_lowercase)

    def __len__(self):
        """
        Calculates the length of the dataset-
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns one sample (dict consisting of an image and its label)
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Read the image and labels
        image_path = os.path.join(self.root_dir, self.data[idx][1])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        # Shape of the image should be H,W,C where C=1
        image = np.expand_dims(image, 0)
        # The label is the index of the class name in the list ['0','1',...,'9','a','b',...'z']
        # because we should have integer labels in the range 0-35 (for 36 classes)
        label = self.class_names.index(self.data[idx][0])
                
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [3]:
sign_lang_dataset = SignLangDataset(csv_file="labels.csv", root_dir=DATASET_PATH)
#print(sign_lang_dataset[1]['label'])
data_len = len(sign_lang_dataset)
train_ratio = 0.8
train_size = int(train_ratio * data_len)
val_size = data_len - train_size
train_dataset, val_dataset = random_split(sign_lang_dataset, [train_size, val_size])

In [4]:
def build_data_for_scikit(dataset):
    X_list = list()
    y_list = list()
    for data in dataset:
        X_list.append(data['image'])
        y_list.append(data['label'])
    return np.array(X_list), np.array(y_list)

In [5]:
X_train, y_train = build_data_for_scikit(train_dataset)
X_train = X_train.reshape(train_size, -1)
X_test, y_test = build_data_for_scikit(val_dataset)
X_test = X_test.reshape(val_size, -1)
print(X_train.shape)
print(X_test.shape)

(7744, 16384)
(1936, 16384)


# Desc Trees

In [9]:
#experimental
clf = DecisionTreeClassifier(criterion='gini')
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities


In [10]:
clfs = []
for i in range(200, len(ccp_alphas), 300):
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alphas[i])
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

Number of nodes in the last tree is: 291 with ccp_alpha: 0.014482925279712844


In [13]:
print(clfs)
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]
print(train_scores)
print(test_scores)

[DecisionTreeClassifier(ccp_alpha=0.00012913223140495868, random_state=0), DecisionTreeClassifier(ccp_alpha=0.00023243801652892576, random_state=0), DecisionTreeClassifier(ccp_alpha=0.0003228305785123967, random_state=0), DecisionTreeClassifier(ccp_alpha=0.0004958677685950414, random_state=0), DecisionTreeClassifier(ccp_alpha=0.0010433475320776988, random_state=0)]
[0.8929493801652892, 0.8607954545454546, 0.7957128099173554, 0.7004132231404959, 0.49599690082644626]
[0.37964876033057854, 0.38016528925619836, 0.37706611570247933, 0.381198347107438, 0.3517561983471074]


In [44]:
tree_sklearn = DecisionTreeClassifier(criterion='gini')
tree_sklearn.fit(X_train, y_train)

DecisionTreeClassifier()

In [46]:
accuracy_train = tree_sklearn.score(X_train, y_train)
accuracy_test = tree_sklearn.score(X_test, y_test)
print(accuracy_train)
print(accuracy_test)

1.0
0.368801652892562


# k nearest neighbour

In [24]:
from sklearn.neighbors import KNeighborsClassifier
for i in range(2,12):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    print(model.score(X_test, y_test))

0.7634297520661157
0.7784090909090909
0.7747933884297521
0.7778925619834711
0.765495867768595
0.7582644628099173
0.7541322314049587
0.7489669421487604
0.737603305785124
0.7396694214876033


# ANN

In [11]:
X_train = X_train / 255
X_test = X_test / 255

In [12]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(100,), random_state=1, verbose=True, max_iter = 30)
clf.fit(X_train, y_train)

Iteration 1, loss = 3.45646219
Iteration 2, loss = 3.25211696


/opt/conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(200,), max_iter=30, random_state=1,
              verbose=True)

In [ ]:
print(clf.score(X_test, y_test))
print(clf.score(X_train, y_train))

0.2815082644628099
